In [1]:
import math
from scipy.stats import rankdata
import pandas as pd

In [102]:
def position_angles(rows, sumdelegates):
    
    Totals = [4, 15, 33, 61, 95] # Threshold for new row. For instance, more than 15 seats means 3 rows
    
    # Maximum radius of spot is 0.5/rows; leave a bit of space.
    radius = 0.4/rows

    # Create list of centre spots
    poslist_24 = []

    for i in range(1, rows):
        """ The relevant math is borrowed from David Richfield (slashme on GitHub)"""
        
        # Each row can contain pi/(2asin(2/(3n+4i-2))) spots, where n is the number of rows and i is the number
        # of the current row. Fill each row proportionally to the "fullness" of the diagram, up to the second-last row.
        
        J = int(float(sumdelegates) / Totals[rows-1] * math.pi/(2*math.asin(2.0/(3.0*rows+4.0*i-2.0))))
        
        # The radius of the ith row in an N-row diagram (Ri) is (3*N+4*i-2)/(4*N)
        
        R = (3.0*rows+4.0*i-2.0)/(4.0*rows)
        
        if J == 1:
            poslist_24.append([math.pi/2.0, 1.75*R, R])
        else:
            for j in range(J):
                # The angle to a spot is n.(pi-2sin(r/Ri))/(Ni-1)+sin(r/Ri) where Ni is the number in the arc
                # x=R.cos(theta) + 1.75
                # y=R.sin(theta)
                
                angle = float(j) * (math.pi-2.0*math.sin(radius/R)) / (float(J)-1.0)+math.sin(radius/R)   
                poslist_24.append([angle, R*math.cos(angle)+1.75, R*math.sin(angle)])

                
    # Now whatever seats are left go into the outside row:
    J_2 = sumdelegates-len(poslist_24)
    R = (7.0*rows-2.0)/(4.0*rows)
    if J_2 == 1:
        poslist_24.append([math.pi/2.0, 1.75*R, R])
    else:
        for j in range(J_2):
            angle = float(j) * (math.pi-2.0*math.sin(radius/R)) / (float(J_2)-1.0)+math.sin(radius/R)
            poslist_24.append([angle, R*math.cos(angle)+1.75, R*math.sin(angle)])
    poslist_24.sort(reverse = True)
    poslist_data = pd.DataFrame(data = poslist_24, columns = ['angle', 'x', 'y'])
    
    return poslist_data

In [275]:
dataframe = pd.read_excel(r'C:\Users\mariu\Documents\Project Local Elections\seat_allocation.xlsx')

poslist_final = pd.DataFrame(columns = ['angle', 'x', 'y'])

dataframe['rows'] = 3
dataframe.loc[dataframe['council_seats'] >= 33,  'rows'] = 4
dataframe.loc[dataframe['council_seats'] >= 61,  'rows'] = 5

for i in range(len(dataframe.city.unique())):
    
    city_now = dataframe.city.unique()[i]
    dataframe_1 = dataframe[(dataframe.city == city_now)].reset_index()
    row = dataframe_1.rows[0]
    no_seats = dataframe_1.council_seats[0]
    
    poslist_final = pd.concat([poslist_final, position_angles(row, no_seats)])

poslist_final

,angle,x,y
0,3.080093,0.128072,0.099874
1,3.068929,0.378628,0.099824
2,3.052821,0.629430,0.099737
3,3.027556,0.880683,0.099566
4,2.828544,0.203977,0.500437
...,...,...,...
45,0.250162,3.324418,0.402286
46,0.114037,2.619317,0.099566
47,0.088772,2.870570,0.099737
48,0.072663,3.121372,0.099824


In [276]:
data_graph = dataframe.join(poslist_final.reset_index())
data_graph

,votes,rank,party,city,council_seats,index_pos,rows,index,angle,x,y
0,23937.914672,1,AB,Amberg,40,1,4,0,3.080093,0.128072,0.099874
1,152948.802362,1,SPD,Amberg,40,2,4,1,3.068929,0.378628,0.099824
2,76474.401181,2,SPD,Amberg,40,2,4,2,3.052821,0.629430,0.099737
3,50982.934121,3,SPD,Amberg,40,2,4,3,3.027556,0.880683,0.099566
4,38237.200591,4,SPD,Amberg,40,2,4,4,2.828544,0.203977,0.500437
...,...,...,...,...,...,...,...,...,...,...,...
1793,43876.163613,16,CSU,Würzburg,50,7,4,45,0.250162,3.324418,0.402286
1794,52768.336401,1,Bf,Würzburg,50,8,4,46,0.114037,2.619317,0.099566
1795,93249.896983,1,WL,Würzburg,50,8,4,47,0.088772,2.870570,0.099737
1796,46624.948491,2,WL,Würzburg,50,8,4,48,0.072663,3.121372,0.099824


In [229]:
data_graph['index_pos'] = 4
data_graph.loc[data_graph['party'] == 'AfD', 'index_pos'] = 9
data_graph.loc[data_graph['party'] == 'SPD', 'index_pos'] = 2
data_graph.loc[data_graph['party'] == 'Linke', 'index_pos'] = 1
data_graph.loc[data_graph['party'] == 'Grüne', 'index_pos'] = 3
data_graph.loc[data_graph['party'] == 'CSU', 'index_pos'] = 7
data_graph.loc[data_graph['party'] == 'FW', 'index_pos'] = 6
data_graph.loc[data_graph['party'] == 'FDP', 'index_pos'] = 5
data_graph.loc[data_graph['party'] == 'BIA', 'index_pos'] = 10
data_graph.loc[data_graph['party'] == 'REP', 'index_pos'] = 10
data_graph.loc[data_graph['party'] == 'BP', 'index_pos'] = 8
data_graph.loc[(data_graph['party'] == 'Bürger für Landshut')|(data_graph['party'] == 'Wählervereinigung LJ')|
               (data_graph['party'] == 'SLU')|(data_graph['party'] == 'Christlicher Rathausblock')|
               (data_graph['party'] == 'Zusammen Bayern')|(data_graph['party'] == 'Pro Ulm')|(data_graph['party'] == 'PEG')|
               (data_graph['party'] == 'Bürger für Penzberg')|(data_graph['party'] == 'Freie Lokalpolitik Penzberg')|
               (data_graph['party'] == 'CSB')|(data_graph['party'] == 'Pro Schweinfurt')|
               (data_graph['party'] == 'Bürgerforum')|(data_graph['party'] == 'WL')|(data_graph['party'] == 'Pro Augsburg')|
               (data_graph['party'] == 'Christlich Soziale Bürger')|
               (data_graph['party'] == 'Pro Coburg')|(data_graph['party'] == 'Bürger für Dachau')|
               (data_graph['party'] == 'Dorfen West')|(data_graph['party'] == 'EWG')|
               (data_graph['party'] == 'TEG')|(data_graph['party'] == 'WAN')|
               (data_graph['party'] == 'Freisinger Mitte')|(data_graph['party'] == 'BBV')|
               (data_graph['party'] == 'Parteifreie')|(data_graph['party'] == 'UCS')|
               (data_graph['party'] == 'WG')|(data_graph['party'] == 'Bündnis für Karlsfeld')
               , 'index_pos'] = 8

data_graph.loc[(data_graph['party'] == 'Amberger Bunte')|(data_graph['party'] == 'AUX')|
               (data_graph['party'] == 'Polit-WG')|(data_graph['party'] == 'V-Partei')|
               (data_graph['party'] == 'Generation Fortschritt')|(data_graph['party'] == 'volt')|
               (data_graph['party'] == 'Future for Kempten')|(data_graph['party'] == 'Frauenliste')|
               (data_graph['party'] == 'Gemeinwohl Schwindkircheng')|(data_graph['party'] == 'Generation KF')|
               (data_graph['party'] == 'FAIR')|(data_graph['party'] == 'mut')|
                (data_graph['party'] == 'RoLi')|(data_graph['party'] == 'Die Guten')|
                (data_graph['party'] == 'politbande')|(data_graph['party'] == 'V3-Partei')|
                (data_graph['party'] == 'Gemeinsam für Gemeinwohl')|(data_graph['party'] == 'Linke Liste')
               , 'index_pos'] = 1

In [277]:
data_graph['color'] = '#999999'

data_graph.loc[data_graph['party'] == 'BP', 'color'] = '#98F5FF'
data_graph.loc[data_graph['party'] == 'Partei', 'color'] = '#B5152B'
data_graph.loc[data_graph['party'] == 'AfD', 'color'] = '#87CEFF'
data_graph.loc[data_graph['party'] == 'ödp', 'color'] = '#698B22'
data_graph.loc[data_graph['party'] == 'FW', 'color'] = '#FFA500'
data_graph.loc[data_graph['party'] == 'Linke', 'color'] = '#A020F0'
data_graph.loc[data_graph['party'] == 'BIA', 'color'] = '#8B4513'
data_graph.loc[(data_graph['party'] != 'ödp') & (data_graph['party'] != 'Partei') & (data_graph['index_pos'] == 4)
               , 'color'] = '#E2D3C8'
data_graph.loc[(data_graph['party'] != 'Linke') & (data_graph['index_pos'] == 1), 'color'] = '#DDA0DD'
data_graph.loc[data_graph['party'] == 'FDP', 'color'] = '#F7F705'
data_graph.loc[data_graph['party'] == 'Grüne', 'color'] = '#00AA00'
data_graph.loc[data_graph['party'] == 'SPD', 'color'] = '#FF0000'
data_graph.loc[data_graph['party'] == 'CSU', 'color'] = '#000000'

In [278]:
#data_graph = pd.read_excel(r'C:\Users\mariu\Documents\Project Local Elections\weird_3.xlsx')
data_graph_1 = data_graph[:896].reset_index()
data_graph_2 = data_graph[896:].reset_index()
data_graph_1, data_graph_2

(     level_0          votes  rank  party      city  council_seats  index_pos  \
 0          0   23937.914672     1     AB    Amberg             40          1   
 1          1  152948.802362     1    SPD    Amberg             40          2   
 2          2   76474.401181     2    SPD    Amberg             40          2   
 3          3   50982.934121     3    SPD    Amberg             40          2   
 4          4   38237.200591     4    SPD    Amberg             40          2   
 ..       ...            ...   ...    ...       ...            ...        ...   
 891      891   27562.146685     5    SPD  Landshut             44          2   
 892      892   22968.455571     6    SPD  Landshut             44          2   
 893      893   19687.247632     7    SPD  Landshut             44          2   
 894      894  157038.999243     1  Grüne  Landshut             44          3   
 895      895   78519.499622     2  Grüne  Landshut             44          3   
 
      rows  index     angl

In [123]:
import networkx as nx
import matplotlib.pyplot as plt
import plotly.offline as pyo
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly import tools
import chart_studio

chart_studio.tools.set_credentials_file(username='marius92', api_key='4naxu3XiGZTGniWO02z1')

In [279]:
# Create individual graphs

fig = go.Figure()

list_parties= data_graph_1 .party.unique()
list_cities = data_graph_1 .city.unique()

for j in range(len(list_cities)):
    
    city_now = list_cities[j]
    data_trace = data_graph_1 [data_graph_1 ['city'].str.match(city_now)]
    list_parties = data_trace.party.unique()
    
    if j == 0:
        for i in range(len(list_parties)):
            
            party_now = list_parties[i]
            
            fig.add_trace(go.Scatter(
                x= data_trace[data_trace['party'].str.match(party_now)]['x'], 
                y= data_trace[data_trace['party'].str.match(party_now)]['y'],
                mode='markers',
                name = party_now,
                marker = dict(
                    size = 18,
                    color = data_trace[data_trace['party'].str.match(list_parties[i])]['color']),
                hoverinfo = 'text',
                hovertext = party_now,
                visible = True, # Change to forecasted politician/or just number of seat for party ?
            )
            )
    
    
    if j != 0:       
    # Generate a scatter of a certain color for every party
        for i in range(len(list_parties)):
            party_now = list_parties[i]
    
            fig.add_trace(go.Scatter(
                x= data_trace[data_trace['party'].str.match(party_now)]['x'], 
                y= data_trace[data_trace['party'].str.match(party_now)]['y'],
                mode='markers',
                name = party_now,
                marker = dict(
                    size = 18,
                    color = data_trace[data_trace['party'].str.match(list_parties[i])]['color']),
                hoverinfo = 'text',
                hovertext = party_now,
                visible = False, # Change to forecasted politician/or just number of seat for party ?
            )
            )
        
    x_annot = (data_trace['x'].iloc[-1] / 2) + data_trace['x'].iloc[0] # x center alignment for text
    y_max = data_trace['y'].max()
        
    fig.layout.update(
        legend = dict(orientation='v', # Put in a legend
                        font = dict(family = "Old Standard TT",
                                    size = 14,
                                    color = 'black',
                                    ),
                        x = -0.15,
                        y = data_graph_1 ['y'].iloc[0] +0.9,
                ),
    )
        
    fig.layout.update(
    xaxis = dict(showgrid=False, zeroline=False, visible = False), # Hide anything related to the x-axis
    yaxis = dict(showgrid=False, zeroline=False, visible = False), # Hide anything related to the y-axis
    plot_bgcolor = 'white',
    title = dict(font = dict(family = 'Old Standard TT',
                             size = 32,
                             color = 'black',
                        ),
                 x = 0.64,
            ),
    )

In [280]:
button_data = data_graph_1.groupby(['city', 'party'], as_index = False).mean()
#utton_data.to_excel(r'C:\Users\mariu\Documents\Project Local Elections\weird_3.xlsx')
button_data

,city,party,level_0,votes,rank,council_seats,index_pos,rows,index,angle,x,y
0,Amberg,AB,0.000000,23937.914672,1.0,40.0,1.0,4.0,0.000000,3.080093,0.128072,0.099874
1,Amberg,CSU,30.000000,50922.540379,10.0,40.0,7.0,4.0,30.000000,0.707874,2.647230,0.751344
2,Amberg,FDP,19.000000,14233.282932,1.0,40.0,5.0,4.0,19.000000,1.570796,1.750000,1.375000
3,Amberg,FW,20.000000,25454.486942,1.0,40.0,6.0,4.0,20.000000,1.570796,1.750000,1.125000
4,Amberg,Grüne,13.000000,31693.116346,2.0,40.0,3.0,4.0,13.000000,2.100109,1.094199,1.110973
...,...,...,...,...,...,...,...,...,...,...,...,...
193,Kulmbach,Grüne,863.000000,22093.296909,1.0,30.0,3.0,3.0,7.000000,2.384314,0.841615,0.858683
194,Kulmbach,SPD,864.111111,27739.400167,5.0,30.0,2.0,3.0,8.111111,2.264234,1.098569,0.426391
195,Landshut,Grüne,894.500000,117779.249433,1.5,44.0,3.0,4.0,8.500000,2.533061,0.828260,0.644922
196,Landshut,Linke,886.000000,21707.937117,1.0,44.0,1.0,4.0,0.000000,3.080093,0.128072,0.099874


In [154]:
table_data = data_graph.groupby(['city', 'party'], as_index = False).max()
table_data.to_excel(r'C:\Users\mariu\Documents\Project Local Elections\data_table.xlsx')

In [94]:
button_data = pd.read_excel(r'C:\Users\mariu\Documents\Project Local Elections\weird_3.xlsx')

In [269]:
button_data_2 = pd.DataFrame()

for i in range(len(button_data.city.unique())):
    
    name = list_cities[i]
    button_data[name] = False
    button_data.loc[button_data['city'] == name, name] = True
    

In [270]:
buttons=[]

for i in range(len(button_data.city.unique())):
    
    name = button_data.city.unique()[i]
    buttons.append(dict(method='update',
                        label= '<b>' +  button_data.city.unique()[i] + '</b>',
                        args=[{'visible': button_data[name].values,
                               'title': " <b>" + button_data.city.unique()[i] + "</b>"}, ],
                        )
                  )


In [281]:
fig.layout.update(
    updatemenus=[
        go.layout.Updatemenu(
            buttons=buttons,
            x = data_graph['x'].iloc[0] -0.05,
            y = 1.24,
            bgcolor = '#efefef',
            font = dict(family = 'Old Standard TT',
                        size = 12,
                       ),
            borderwidth = 1.2,
        )
    ])

fig['layout'].update(width = 750, height = 500)
fig.update_xaxes(automargin=True)
filename = 'Parliament_seats_1'
py.iplot(fig, filename = filename)

In [282]:
data_overall = data_graph.groupby(['city']).max()
data_overall['council_seats'].sum()

1798